In [ ]:
import pandas as pd
from matplotlib import pyplot
from pandas.plotting import autocorrelation_plot
from statsmodels.tsa.stattools import adfuller
from statsmodels.tsa.arima.model import ARIMA
import statsmodels
import statsmodels.api as sm
from statsmodels.graphics.tsaplots import plot_acf
import numpy as np
from statsmodels.graphics.tsaplots import plot_predict

In [ ]:
df = pd.read_csv('load_power.csv', usecols=['start', 'mean'])
df['start'] = pd.to_datetime(df['start'], format='%Y-%m-%d %H:%M:%S')
df = df.set_index('start')
# df = df.asfreq('H')
df = df[['mean']].tail(4*7*24)

def split_dataset(data):
    train, validate, test = data[0:-24*60], data[-24*60:-24*14], data[-24*14:0]
    return train, validate, test

train, validate, test = split_dataset(df)
# first_diff = (df['mean'] - df['mean'].shift(1)).dropna()
# first_diff = df['mean'].diff().dropna()
# rolling_mean = df.rolling(window = 12).mean()
# df['rolling_mean_diff'] = df - df.shift(24)
# df['input'] = df['mean'].tail(4*7*24)
# df['rolling_mean_diff'].plot()
# df.head(24*7).plot()
# print(df.head())
# print(first_diff.head())
# first_diff.head(24*7).plot()
# print(adfuller(df['mean']))
# print(adfuller((df['mean'] - df['mean'].shift(2)).dropna()))
# print(train)
# pyplot.plot(train)
# plot_acf(first_diff)
# values = df['mean'].diff().dropna().to_numpy()
# df.plot()
# first_diff.tail(24*2).plot()
# model = ARIMA(first_diff, order=(1, 1, 1))
# fit = model.fit()
# fig, ax = pyplot.subplots()
# ax = first_diff.plot(ax=ax)
# plot_predict(fit, 6*24, 8*24, dynamic=True, ax=ax)
# pyplot.show()
# df['first_diff']  =first_diff

# model=sm.tsa.statespace.SARIMAX(df['mean'], order=(1, 1, 1),seasonal_order=(1,1,1,24))
# results=model.fit()
start = 2*7*24 
# train_period = 3*7*24
model = sm.tsa.statespace.SARIMAX(df['mean'][:start], order=(1, 1, 1),seasonal_order=(1,1,1,24))
results = model.fit()
df["pred"] = np.nan
df["pred_u"] = np.nan
df["pred_l"] = np.nan
for x in range(14):
    slice_start = start + 24 * x
    slice_end = slice_start + 24
    # train_start = x * 7 * 24
    # train_end = train_start + train_period
    pred = results.get_prediction(slice_start, slice_end - 1, dynamic=True)
    df["pred"].iloc[slice_start:slice_end] = pred.predicted_mean
    # df.iloc[slice_start:slice_end].loc[:, f"pred"] = pred.predicted_mean
    conf_int = pred.conf_int(alpha=0.8)
    # df.update(conf_int)
    df["pred_u"].iloc[slice_start:slice_end] = conf_int['upper mean']
    df["pred_l"].iloc[slice_start:slice_end] = conf_int['lower mean']
    # df.iloc[slice_start:slice_end].loc[:, f"pred_u"] = conf_int['upper mean']
    # df.iloc[slice_start:slice_end].loc[:, f"pred_l"] = conf_int['lower mean']
    results = results.append(df['mean'][slice_start:slice_end])

# df['forecast']=results.predict(4*7*24-3*24, 4*7*24 ,dynamic=24)
# df['c'] = df['mean'].tail(3*24)
# df[start:].plot(figsize=(12,8))
# fig, ax = pyplot.subplots()
# ax = df[4*7*24-4*24:].plot(ax=ax)
# plot_predict(results, 4*7*24-3*24, 4*7*24, ax=ax)
# pyplot.show()

In [ ]:
# for x in range(7):
#     slice_start = start + 24 * x
#     slice_end = slice_start + 24
#     print(f'{x}: {df["mean"][slice_start:slice_end].sum()} {df[f"d{x}"].sum()}')

pyplot.figure()
fig, ax = pyplot.subplots(figsize=(15, 5))
df['mean'][start:].plot(ax=ax)
df['pred'][start:].plot(ax=ax)
# df['pred_l'][start:].plot()
# df['pred_u'][start:].plot()

ax.fill_between(df.index, df['pred_l'], df['pred_u'], color='k', alpha=0.1);    
pyplot.legend()
pyplot.show()